
# Llama fine-tuning with Ludwig


Voir https://ludwig.ai/latest/ et https://colab.research.google.com/drive/1Ly01S--kUwkKQalE-75skalp-ftwl0fE?usp=sharing

Il faut obtenir les droits sur Llama : https://ai.meta.com/resources/models-and-libraries/llama-downloads/

Puis les demander sur HF : https://huggingface.co/meta-llama/Llama-2-7b-hf


In [1]:
# !pip install "ludwig[llm]" --quiet

In [2]:
from decouple import Config, RepositoryEnv
import os
DOTENV_FILE = '../.env'
env_config = Config(RepositoryEnv(DOTENV_FILE))
os.environ["HUGGING_FACE_HUB_TOKEN"] = env_config('HUGGING_FACE_HUB_TOKEN')

In [3]:
import yaml

config_str = """
model_type: llm
base_model: Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB
#base_model: meta-llama/Llama-2-7b-hf

quantization:
  bits: 4

adapter:
  type: lora

prompt:
  template: |
    ### Instruction:
    {instruction}

    ### Input:
    {input}

    ### Response:

input_features:
  - name: prompt
    type: text

output_features:
  - name: output
    type: text

trainer:
  type: finetune
  learning_rate: 0.0001
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 1
  learning_rate_scheduler:
    warmup_fraction: 0.01

preprocessing:
  sample_ratio: 0.1
"""

config = yaml.safe_load(config_str)

In [4]:
import logging
from ludwig.api import LudwigModel


model = LudwigModel(config=config, logging_level=logging.INFO)
results = model.train(dataset="ludwig://alpaca")
print(results)

/media/data-nvme/dev/src/llm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Created a temporary directory at /tmp/tmpcy2fi60r
Writing /tmp/tmpcy2fi60r/_remote_module_non_scriptable.py



╒════════════════════════╕
│ EXPERIMENT DESCRIPTION │
╘════════════════════════╛

╒══════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                            │
├──────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                       │
├──────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /media/data-nvme/dev/src/llm/llama-cpp-python/results/api_experiment_run_5                │
├──────────────────┼───────────────────────────────────────────────────────────────────────────────────────────┤
│ ludwig_version   │ '0.8.2'                                                                                   │
├────────────

No sha256 digest provided for dataset alpaca, cannot verify.
Contents:
    alpaca_data.json: 2eddafc6b977608d778aaab8dfc7e50e547b3af9826dfb9e909d9fc362e4a419
Found cached dataset and meta.json with the same filename of the dataset, but checksums don't match, if saving of processed input is not skipped they will be overridden
Using full dataframe
Building dataset (it may take a while)


Loaded HuggingFace implementation of Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB tokenizer


...

We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 3/3 [00:15<00:00,  5.25s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB and are newly initialized: ['model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_fr

Done.


Loaded HuggingFace implementation of Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB tokenizer
No padding token id found. Using eos_token as pad_token.
Trainable Parameter Summary For Fine-Tuning
Fine-tuning with adapter: lora
trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199
Tuning batch size...
Tuning batch size...
Exploring batch_size=1
Throughput at batch_size=1: 0.40539 samples/s
Exploring batch_size=2
Throughput at batch_size=2: 0.33568 samples/s
Throughput decrease at batch_size=2
Selected batch_size=1

╒══════════╕
│ TRAINING │
╘══════════╛

Creating fresh model training run.
Training for 3640 step(s), approximately 1 epoch(s).
Early stopping policy: 5 round(s) of evaluation, or 18200 step(s), approximately 5 epoch(s).

Starting with step 0, epoch: 0
Training:   9%|▉         | 321/3640 [04:48<48:15,  1.15it/s, loss=0.194]   

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 10.91 GiB total capacity; 9.15 GiB already allocated; 107.81 MiB free; 9.62 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF